In [1]:
import numpy as np
import pandas as pd
import torch
from WideAndDeep_pytorch.wide_deep import WideAndDeepModel
from WideAndDeep_pytorch.avazu import AvazuDataset
from WideAndDeep_pytorch.train import train,test,EarlyStopper
from torch.utils.data import DataLoader
from IPython.core.interactiveshell import  InteractiveShell
InteractiveShell.ast_node_interactivity='all'
pd.set_option('max_columns',600)
pd.set_option('max_rows',500)
torch.manual_seed(0)

# load dataset
#dataset=AvazuDataset('./data/train_150m.csv',rebuild_cache=False)
dataset=AvazuDataset('./data/train_toy.csv',rebuild_cache=False) # reach out the author to get small/big dataset 
model=WideAndDeepModel(dataset.field_dims, embed_dim=16,mlp_dims=(16, 16), dropout=0.2)


#split dataset into train/test
train_length = int(len(dataset) * 0.9)
valid_length = int(len(dataset) * 0.1)
print("train_length,valid_length",train_length,valid_length)
test_length = len(dataset) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_length, valid_length, test_length))

#DataLoader
train_data_loader = DataLoader(train_dataset, batch_size=256, num_workers=0)
valid_data_loader = DataLoader(valid_dataset, batch_size=256, num_workers=0)
test_data_loader = DataLoader(test_dataset, batch_size=256, num_workers=0)



criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.000001)

# Training
train_auc = []
test_auc = []

for epoch_i in range(10):
    device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
    #device = None
    train(model, optimizer, train_data_loader, criterion, device=device)
    auc_train = test(model, train_data_loader, device=device) # test
    auc_valid = test(model, valid_data_loader, device=device)
    #auc_test = test(model, test_data_loader, device=None)
    print('epoch:{}：'.format(epoch_i))
    print('Train AUC:{}'.format(auc_train))
    print('Test AUC:{}'.format(auc_valid))
    train_auc.append(auc_train)
    test_auc.append(auc_valid)

print('train_auc',train_auc)
print('test_auc',test_auc)

Model Architecture
Sequential(
  (0): Linear(in_features=352, out_features=16, bias=True)
  (1): ReLU()
  (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.2, inplace=False)
  (4): Linear(in_features=16, out_features=16, bias=True)
  (5): ReLU()
  (6): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.2, inplace=False)
  (8): Linear(in_features=16, out_features=1, bias=True)
)
train_length,valid_length 899 99


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 437.41it/s]


epoch:0：
Train AUC:0.5973978919631093
Test AUC:0.5144736842105263


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 319.93it/s]


epoch:1：
Train AUC:0.5982072275550536
Test AUC:0.5144736842105263


...

epoch:7：
Train AUC:0.6094814605684171
Test AUC:0.5223684210526316


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 334.42it/s]


epoch:8：
Train AUC:0.6154573687182383
Test AUC:0.5335526315789474


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 401.45it/s]

epoch:9：
Train AUC:0.6237107095802749
Test AUC:0.5388157894736842
train_auc [0.5973978919631093, 0.5982072275550536, 0.5989412761151891, 0.5999200075287032, 0.6012469414643327, 0.6025079992471297, 0.6053124411820064, 0.6094814605684171, 0.6154573687182383, 0.6237107095802749]
test_auc [0.5144736842105263, 0.5144736842105263, 0.5151315789473685, 0.5157894736842106, 0.5164473684210527, 0.5171052631578947, 0.5190789473684211, 0.5223684210526316, 0.5335526315789474, 0.5388157894736842]
